<a href="https://colab.research.google.com/github/PapaBNdiaye/STATS-INFERENTIELLE/blob/main/MesFonctionsTests_Statistiques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
###########################################
# Chargement des packages nécessaires #####
###########################################
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

###########################################################
# Définition des fonctions de tests statistiques #############
###########################################################

def test_t(group1, group2):
    """
    Effectue un test t pour comparer les moyennes de deux groupes indépendants.
    Vérifie les hypothèses de normalité et d'égalité des variances.
    Utilise le test de Welch si les variances sont inégales.
    Capture les erreurs éventuelles lors des tests de normalité et de variances.

    Args:
        group1 (array-like): Premier groupe de données.
        group2 (array-like): Deuxième groupe de données.

    Affiche:
        Les résultats des tests de normalité et d'égalité des variances.
        La statistique t et la valeur p du test approprié (Student ou Welch).
    """
    # Test de normalité
    try:
        _, p1 = stats.shapiro(group1)
        _, p2 = stats.shapiro(group2)
        print(f"Test de normalité (Shapiro-Wilk) :")
        print(f"  Groupe 1 : p = {p1:.4f}")
        print(f"  Groupe 2 : p = {p2:.4f}")
        if p1 < 0.05 or p2 < 0.05:
            print("  Attention : Les données ne semblent pas suivre une distribution normale.")
            print("  Considérez l'utilisation d'un test non paramétrique comme le test de Mann-Whitney U.")
    except ValueError as e:
        print(f"Erreur dans le test de normalité : {e}")
        return

    # Test d'égalité des variances
    try:
        _, p_var = stats.levene(group1, group2)
        print(f"\nTest d'égalité des variances (Levene) :")
        print(f"  p = {p_var:.4f}")
        if p_var < 0.05:
            print("  Les variances des deux groupes semblent significativement différentes.")
            print("  Le test de Welch sera utilisé.")
        else:
            print("  Les variances des deux groupes ne semblent pas significativement différentes.")
            print("  Le test t de Student classique sera utilisé.")
    except ValueError as e:
        print(f"Erreur dans le test de variances : {e}")
        return

    # Test t (Student ou Welch)
    equal_var = p_var >= 0.05
    t_stat, p_value = stats.ttest_ind(group1, group2, equal_var=equal_var)
    test_name = "Test t de Student" if equal_var else "Test t de Welch"
    print(f"\n{test_name} :")
    print(f"  t = {t_stat:.4f}")
    print(f"  p = {p_value:.4f}")

    # Interprétation
    alpha = 0.05
    if p_value < alpha:
        print(f"\nConclusion : Rejet de l'hypothèse nulle au seuil α = {alpha}.")
        print("Il y a une différence significative entre les moyennes des deux groupes.")
    else:
        print(f"\nConclusion : Non-rejet de l'hypothèse nulle au seuil α = {alpha}.")
        print("Il n'y a pas de différence significative entre les moyennes des deux groupes.")

#############################################################
# definition de la fonction de test de Mann-Whitney U #######
#############################################################

def test_mannwhitney(group1, group2):
    """
    Effectue un test de Mann-Whitney U pour comparer deux groupes indépendants.
    Utilisé lorsque les hypothèses de normalité ne sont pas respectées.
    Capture les erreurs éventuelles lors du test.

    Args:
        group1 (array-like): Premier groupe de données.
        group2 (array-like): Deuxième groupe de données.

    Affiche:
        La statistique du test et la valeur p.
    """
    try:
        stat, p_value = stats.mannwhitneyu(group1, group2, alternative='two-sided')
    except ValueError as e:
        print(f"Erreur lors du test de Mann-Whitney : {e}")
        return

    print(f"Test de Mann-Whitney U : stat = {stat:.4f}, p = {p_value:.4f}")

##############################################################
# definition de la fonction de test de l'ANOVA ###############
##############################################################

def test_anova(data, formula):
    """
    Effectue une analyse de variance (ANOVA) à un facteur.
    Vérifie l'hypothèse de normalité des résidus et d'égalité des variances.
    Capture les erreurs éventuelles lors de la création du modèle et du calcul des résidus.

    Args:
        data (pandas.DataFrame): Le dataframe contenant les données.
        formula (str): La formule décrivant le modèle (ex: 'y ~ x').

    Affiche:
        La table ANOVA.
    """
    try:
        model = ols(formula, data=data).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
    except Exception as e:
        print(f"Erreur lors de l'ANOVA : {e}")
        return

    _, p_resid = stats.shapiro(model.resid)
    if p_resid < 0.05:
        print("Attention : Les résidus ne semblent pas suivre une distribution normale.")

    dependent_var = formula.split('~')[0].strip()
    independent_var = formula.split('~')[1].strip()

    try:
        _, p_var = stats.levene(*[data.loc[data[independent_var] == level, dependent_var] for level in data[independent_var].unique()])
    except ValueError as e:
        print(f"Erreur lors du test de variances : {e}")
        return

    if p_var < 0.05:
        print("Attention : Les variances des groupes semblent différentes.")

    print("ANOVA :")
    print(anova_table)


##############################################################
# definition de la fonction de test de Kruskal-Wallis ########
##############################################################

def test_kruskal(*args):
    """
    Effectue un test de Kruskal-Wallis pour comparer plusieurs groupes indépendants.
    Utilisé lorsque les hypothèses de normalité ne sont pas respectées.
    Capture les erreurs éventuelles lors du test.

    Args:
        *args: Groupes de données à comparer.

    Affiche:
        La statistique H et la valeur p du test.
    """
    try:
        h_stat, p_value = stats.kruskal(*args)
    except ValueError as e:
        print(f"Erreur lors du test de Kruskal-Wallis : {e}")
        return

    print(f"Test de Kruskal-Wallis : H = {h_stat:.4f}, p = {p_value:.4f}")


#############################################################
# definition de la fonction de test du chi-carré ############
#############################################################

def test_chi2(data, var1, var2):
    """
    Effectue un test du chi-carré d'indépendance.
    Vérifie que les effectifs théoriques sont suffisants.

    Args:
        data (pandas.DataFrame): Le dataframe contenant les données.
        var1 (str): Nom de la première variable catégorielle.
        var2 (str): Nom de la deuxième variable catégorielle.

    Affiche:
        La statistique du chi-carré et la valeur p.
    """
    contingency_table = pd.crosstab(data[var1], data[var2])
    chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)

    if (expected < 5).any():
        print("Attention : Certains effectifs théoriques sont inférieurs à 5, les résultats peuvent ne pas être fiables.")

    print(f"Test du chi-carré : chi2 = {chi2:.4f}, p = {p_value:.4f}")

#############################################################
# definition de la fonction de test exact de Fisher #########
#############################################################

def test_chi2(data, var1, var2):
    """
    Effectue un test du chi-carré d'indépendance.
    Vérifie que les effectifs théoriques sont suffisants.
    Capture les erreurs éventuelles lors du calcul de la table de contingence ou du test.

    Args:
        data (pandas.DataFrame): Le dataframe contenant les données.
        var1 (str): Nom de la première variable catégorielle.
        var2 (str): Nom de la deuxième variable catégorielle.

    Affiche:
        La statistique du chi-carré et la valeur p.
    """
    try:
        contingency_table = pd.crosstab(data[var1], data[var2])
        chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)
    except ValueError as e:
        print(f"Erreur lors du test du chi-carré : {e}")
        return

    if (expected < 5).any():
        print("Attention : Certains effectifs théoriques sont inférieurs à 5, les résultats peuvent ne pas être fiables.")

    print(f"Test du chi-carré : chi2 = {chi2:.4f}, p = {p_value:.4f}")


#############################################################
# definition de la fonction de test exact de Fisher #########
#############################################################

def test_fisher(data, var1, var2):
    """
    Effectue un test exact de Fisher.
    Utilisé pour les petits échantillons.
    Capture les erreurs éventuelles lors du calcul de la table de contingence ou du test.

    Args:
        data (pandas.DataFrame): Le dataframe contenant les données.
        var1 (str): Nom de la première variable catégorielle.
        var2 (str): Nom de la deuxième variable catégorielle.

    Affiche:
        L'odds ratio et la valeur p du test.
    """
    try:
        contingency_table = pd.crosstab(data[var1], data[var2])
        odds_ratio, p_value = stats.fisher_exact(contingency_table)
    except ValueError as e:
        print(f"Erreur lors du test exact de Fisher : {e}")
        return

    print(f"Test exact de Fisher : odds ratio = {odds_ratio:.4f}, p = {p_value:.4f}")


#############################################################
# definition de la fonction de test de Pearson ##############
#############################################################

def test_pearson(x, y):
    """
    Calcule la corrélation de Pearson entre deux variables.
    Vérifie l'hypothèse de normalité des données.
    Capture les erreurs éventuelles lors du test de normalité ou du calcul de la corrélation.

    Args:
        x (array-like): Première série de données.
        y (array-like): Deuxième série de données.

    Affiche:
        Le coefficient de corrélation r et la valeur p.
    """
    try:
        _, p1 = stats.shapiro(x)
        _, p2 = stats.shapiro(y)
    except ValueError as e:
        print(f"Erreur dans le test de normalité : {e}")
        return

    if p1 < 0.05 or p2 < 0.05:
        print("Attention : Les données ne semblent pas suivre une distribution normale.")
        print("Considérez l'utilisation de la corrélation de Spearman.")

    try:
        r, p_value = stats.pearsonr(x, y)
    except ValueError as e:
        print(f"Erreur dans le calcul de la corrélation de Pearson : {e}")
        return

    print(f"Corrélation de Pearson : r = {r:.4f}, p = {p_value:.4f}")


#############################################################
# definition de la fonction de test de Spearman #############
#############################################################

def test_spearman(x, y):
    """
    Calcule la corrélation de Spearman entre deux variables.
    Utilisée lorsque les hypothèses de normalité ne sont pas respectées.
    Capture les erreurs éventuelles lors du calcul de la corrélation.

    Args:
        x (array-like): Première série de données.
        y (array-like): Deuxième série de données.

    Affiche:
        Le coefficient de corrélation rho et la valeur p.
    """
    try:
        rho, p_value = stats.spearmanr(x, y)
    except ValueError as e:
        print(f"Erreur dans le calcul de la corrélation de Spearman : {e}")
        return

    print(f"Corrélation de Spearman : rho = {rho:.4f}, p = {p_value:.4f}")


##############################################################
# Exemple d'utilisation des fonctions ########################
##############################################################

#######################################################
# Charger des datasets afin de tester nos fonctions ###
#######################################################
# Ici je charge les datasets depuis leur URL ##
###############################################
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv') # est aussi disponible sous sklearn
mtcars = pd.read_csv('https://gist.githubusercontent.com/ZeccaLehn/4e06d2575eb9589dbe8c365d61cb056c/raw/64f1660f38ef523b2a1a13be77b002b98665cdfe/mtcars.csv')

print("***********************************************************")
print("******** Datasets chargés avec succès. ********************")
print("***********************************************************")

# Test t de Student
test_t(iris[iris['species'] == 'setosa']['petal_length'],
              iris[iris['species'] == 'versicolor']['petal_length'])

# Test de Mann-Whitney U
test_mannwhitney(mtcars[mtcars['am'] == 0]['mpg'],
                  mtcars[mtcars['am'] == 1]['mpg'])

# ANOVA
test_anova(iris, 'petal_length ~ species')

# Test de Kruskal-Wallis
test_kruskal(iris[iris['species'] == 'setosa']['petal_length'],
              iris[iris['species'] == 'versicolor']['petal_length'],
              iris[iris['species'] == 'virginica']['petal_length'])

# Test du chi-carré
test_chi2(mtcars, 'am', 'vs')

# Test exact de Fisher
test_fisher(mtcars, 'am', 'vs')

# Corrélation de Pearson
test_pearson(mtcars['mpg'], mtcars['wt'])

# Corrélation de Spearman
test_spearman(mtcars['mpg'], mtcars['wt'])

print("***********************************************************")
print("********** Tests effectués avec succès. *******************")
print("***********************************************************")

***********************************************************
******** Datasets chargés avec succès. ********************
***********************************************************
Test de normalité (Shapiro-Wilk) :
  Groupe 1 : p = 0.0548
  Groupe 2 : p = 0.1585

Test d'égalité des variances (Levene) :
  p = 0.0000
  Les variances des deux groupes semblent significativement différentes.
  Le test de Welch sera utilisé.

Test t de Welch :
  t = -39.4927
  p = 0.0000

Conclusion : Rejet de l'hypothèse nulle au seuil α = 0.05.
Il y a une différence significative entre les moyennes des deux groupes.
Test de Mann-Whitney U : stat = 42.0000, p = 0.0019
Attention : Les résidus ne semblent pas suivre une distribution normale.
Attention : Les variances des groupes semblent différentes.
ANOVA :
            sum_sq     df            F        PR(>F)
species   437.1028    2.0  1180.161182  2.856777e-91
Residual   27.2226  147.0          NaN           NaN
Test de Kruskal-Wallis : H = 130.4110, p = 0.